In [1]:
import os
import datetime
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import seaborn as sns
from pathlib import Path
import energizer

In [9]:
#df = energizer.io.read_all_files()
df = energizer.io.read_all_files(hexagons=True, resolution='hex_res_7')
#df = energizer.io.read_all_files(hexagons=True, resolution=8)

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8749277 entries, 0 to 8749276
Data columns (total 27 columns):
 #   Column                      Dtype         
---  ------                      -----         
 0   trip_id                     int64         
 1   taxi_id                     int64         
 2   trip_start_timestamp        datetime64[ns]
 3   trip_end_timestamp          datetime64[ns]
 4   trip_seconds                float64       
 5   trip_miles                  float64       
 6   fare                        float64       
 7   tips                        float64       
 8   tolls                       float64       
 9   extras                      float64       
 10  trip_total                  float64       
 11  payment_type                object        
 12  pickup_census_tract         float64       
 13  pickup_community_area       float64       
 14  pickup_centroid_latitude    float64       
 15  pickup_centroid_longitude   float64       
 16  pickup_centroid_lo

In [20]:
df

,trip_id,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,fare,tips,tolls,extras,trip_total,payment_type,pickup_census_tract,pickup_community_area,pickup_centroid_latitude,pickup_centroid_longitude,pickup_centroid_location,dropoff_community_area,dropoff_centroid_latitude,dropoff_centroid_longitude,dropoff_centroid_location,dropoff_census_tract,company,humidity,temperature,hex_ID_pickup,hex_ID_dropoff
0,0,0,2013-01-01 00:15:00,2013-01-01 00:30:00,480.0,3.8,10.65,2.0,0.0,1.0,13.65,Credit Card,1.703108e+10,8.0,41.892042,-87.631864,"{'coordinates': [-87.6318639497, 41.8920421365...",7.0,41.921877,-87.664078,"{'coordinates': [-87.6640782395, 41.9218774613...",1.703183e+10,None,64.0,-0.19,882664c1e7fffff,882664c123fffff
1,1,1,2013-01-01 00:15:00,2013-01-01 00:30:00,1080.0,2.6,12.85,0.0,0.0,0.0,12.85,Cash,1.703124e+10,24.0,41.892658,-87.652534,"{'coordinates': [-87.6525344838, 41.8926581076...",6.0,41.942585,-87.656644,"{'coordinates': [-87.6566440918, 41.9425851797...",1.703106e+10,Dispatch Taxi Affiliation,64.0,-0.19,882664cad1fffff,882664c163fffff
2,2,2,2013-01-01 00:15:00,2013-01-01 00:15:00,180.0,0.1,4.25,0.0,0.0,1.5,5.75,Cash,1.703108e+10,8.0,41.892073,-87.628874,"{'coordinates': [-87.6288741572, 41.8920726347...",32.0,41.880994,-87.632746,"{'coordinates': [-87.6327464887, 41.8809944707...",1.703184e+10,Dispatch Taxi Affiliation,64.0,-0.19,882664c1e1fffff,882664c1a9fffff
3,3,3,2013-01-01 00:15:00,2013-01-01 00:30:00,780.0,0.9,8.25,0.0,0.0,1.0,9.25,Cash,1.703108e+10,8.0,41.907492,-87.635760,"{'coordinates': [-87.6357600901, 41.9074919303...",8.0,41.907492,-87.635760,"{'coordinates': [-87.6357600901, 41.9074919303...",1.703108e+10,Dispatch Taxi Affiliation,64.0,-0.19,882664c13bfffff,882664c13bfffff
4,4,4,2013-01-01 00:15:00,2013-01-01 00:15:00,480.0,1.8,7.45,0.0,0.0,0.0,7.45,Cash,1.703108e+10,8.0,41.900221,-87.629105,"{'coordinates': [-87.6291051864, 41.9002212967...",32.0,41.880994,-87.632746,"{'coordinates': [-87.6327464887, 41.8809944707...",1.703184e+10,Dispatch Taxi Affiliation,64.0,-0.19,882664c1edfffff,882664c1a9fffff
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8749272,8749272,581,2013-12-31 23:15:00,2013-12-31 23:15:00,840.0,4.8,13.05,0.0,0.0,1.0,14.05,Cash,1.703133e+10,33.0,41.859350,-87.617358,"{'coordinates': [-87.6173580061, 41.859349715]...",8.0,41.904935,-87.649907,"{'coordinates': [-87.6499072264, 41.9049353016...",1.703184e+10,None,89.0,-11.00,882664c1b1fffff,882664c137fffff
8749273,8749273,1131,2013-12-31 23:15:00,2013-12-31 23:45:00,1260.0,4.0,12.25,0.0,0.0,0.0,12.25,Cash,1.703183e+10,7.0,41.921877,-87.664078,"{'coordinates': [-87.6640782395, 41.9218774613...",8.0,41.892508,-87.626215,"{'coordinates': [-87.6262149064, 41.8925077809...",1.703108e+10,None,89.0,-11.00,882664c123fffff,882664c1e1fffff
8749274,8749274,2509,2013-12-31 23:15:00,2013-12-31 23:30:00,840.0,2.8,9.85,0.0,0.0,2.0,11.85,Cash,1.703108e+10,8.0,41.895033,-87.619711,"{'coordinates': [-87.6197106717, 41.8950334495...",24.0,41.908379,-87.670945,"{'coordinates': [-87.6709450752, 41.9083786693...",1.703124e+10,None,89.0,-11.00,882664c1ebfffff,882664cac1fffff
8749275,8749275,451,2013-12-31 23:15:00,2013-12-31 23:30:00,1200.0,2.5,10.05,3.0,0.0,0.0,13.05,Credit Card,1.703106e+10,6.0,41.953400,-87.646007,"{'coordinates': [-87.6460070664, 41.9534000435...",7.0,41.921855,-87.646211,"{'coordinates': [-87.6462109769, 41.9218549112...",1.703107e+10,None,89.0,-11.00,882664c145fffff,882664c107fffff


# Group by Census Tracts

In [12]:
# 1 Hour = 60 Mins
df_grouped_60 = df.groupby([pd.Grouper(key="pickup_census_tract"),
                        pd.Grouper(key="trip_start_timestamp", freq='60Min')]).size()
df_grouped_60 = df_grouped_60.reset_index().rename(columns={0: "trip_count"})

In [13]:
# 2 Hours = 120 Mins
df_grouped_120 = df.groupby([pd.Grouper(key="pickup_census_tract"),
                        pd.Grouper(key="trip_start_timestamp", freq='120Min')]).size()
df_grouped_120 = df_grouped_120.reset_index().rename(columns={0: "trip_count"})

In [14]:
# 6 Hour = 360 Mins
df_grouped_360 = df.groupby([pd.Grouper(key="pickup_census_tract"),
                        pd.Grouper(key="trip_start_timestamp", freq='360Min')]).size()
df_grouped_360 = df_grouped_360.reset_index().rename(columns={0: "trip_count"})

In [15]:
# 24 Hour = 1440 Mins
df_grouped_1440 = df.groupby([pd.Grouper(key="pickup_census_tract"),
                        pd.Grouper(key="trip_start_timestamp", freq='1440Min')]).size()
df_grouped_1440 = df_grouped_1440.reset_index().rename(columns={0: "trip_count"})

In [16]:
df_grouped_60.to_parquet(Path("../../data/output/grouped/df_grouped_60.parquet"))
df_grouped_120.to_parquet(Path("../../data/output/grouped/df_grouped_120.parquet"))
df_grouped_360.to_parquet(Path("../../data/output/grouped/df_grouped_360.parquet"))
df_grouped_1440.to_parquet(Path("../../data/output/grouped/df_grouped_1440.parquet"))

In [17]:
df_grouped_60

,pickup_census_tract,trip_start_timestamp,trip_count
0,1.703101e+10,2013-04-21 01:00:00,1
1,1.703101e+10,2013-02-16 19:00:00,1
2,1.703101e+10,2013-03-01 06:00:00,1
3,1.703101e+10,2013-03-01 20:00:00,1
4,1.703101e+10,2013-06-14 19:00:00,2
...,...,...,...
547892,1.703198e+10,2013-12-31 19:00:00,5
547893,1.703198e+10,2013-12-31 20:00:00,10
547894,1.703198e+10,2013-12-31 21:00:00,7
547895,1.703198e+10,2013-12-31 22:00:00,4


# Group by Hexagons

## Hexagon resolution 7

In [11]:
# 1 Hour = 60 Mins
df_grouped_60_hex_7 = df.groupby([pd.Grouper(key="hex_ID_pickup"),
                        pd.Grouper(key="trip_start_timestamp", freq='60Min')]).size()
df_grouped_60_hex_7 = df_grouped_60_hex_7.reset_index().rename(columns={0: "trip_count"})

In [12]:
# 2 Hours = 120 Mins
df_grouped_120_hex_7 = df.groupby([pd.Grouper(key="hex_ID_pickup"),
                        pd.Grouper(key="trip_start_timestamp", freq='120Min')]).size()
df_grouped_120_hex_7 = df_grouped_120_hex_7.reset_index().rename(columns={0: "trip_count"})

In [13]:
# 6 Hour = 360 Mins
df_grouped_360_hex_7 = df.groupby([pd.Grouper(key="hex_ID_pickup"),
                        pd.Grouper(key="trip_start_timestamp", freq='360Min')]).size()
df_grouped_360_hex_7 = df_grouped_360_hex_7.reset_index().rename(columns={0: "trip_count"})

In [14]:
# 24 Hour = 1440 Mins
df_grouped_1440_hex_7 = df.groupby([pd.Grouper(key="hex_ID_pickup"),
                        pd.Grouper(key="trip_start_timestamp", freq='1440Min')]).size()
df_grouped_1440_hex_7 = df_grouped_1440_hex_7.reset_index().rename(columns={0: "trip_count"})

In [15]:
df_grouped_60_hex_7.to_parquet(Path("../../data/output/grouped/df_grouped_60_hex_7.parquet"))
df_grouped_120_hex_7.to_parquet(Path("../../data/output/grouped/df_grouped_120_hex_7.parquet"))
df_grouped_360_hex_7.to_parquet(Path("../../data/output/grouped/df_grouped_360_hex_7.parquet"))
df_grouped_1440_hex_7.to_parquet(Path("../../data/output/grouped/df_grouped_1440_hex_7.parquet"))

In [16]:
df_grouped_60_hex_7

,hex_ID_pickup,trip_start_timestamp,trip_count
0,872664520ffffff,2013-08-12 13:00:00,2
1,872664521ffffff,2013-01-01 09:00:00,1
2,872664521ffffff,2013-01-01 10:00:00,1
3,872664521ffffff,2013-01-01 11:00:00,4
4,872664521ffffff,2013-01-01 12:00:00,6
...,...,...,...
145505,87275936bffffff,2013-12-16 11:00:00,1
145506,87275936bffffff,2013-12-20 15:00:00,1
145507,87275936dffffff,2013-03-17 01:00:00,1
145508,87275936dffffff,2013-05-21 16:00:00,1


## Hexagon resolution 8

In [3]:
# 1 Hour = 60 Mins
df_grouped_60_hex_8 = df.groupby([pd.Grouper(key="hex_ID_pickup"),
                        pd.Grouper(key="trip_start_timestamp", freq='60Min')]).size()
df_grouped_60_hex_8 = df_grouped_60_hex_8.reset_index().rename(columns={0: "trip_count"})

In [4]:
# 2 Hours = 120 Mins
df_grouped_120_hex_8 = df.groupby([pd.Grouper(key="hex_ID_pickup"),
                        pd.Grouper(key="trip_start_timestamp", freq='120Min')]).size()
df_grouped_120_hex_8 = df_grouped_120_hex_8.reset_index().rename(columns={0: "trip_count"})

In [5]:
# 6 Hour = 360 Mins
df_grouped_360_hex_8 = df.groupby([pd.Grouper(key="hex_ID_pickup"),
                        pd.Grouper(key="trip_start_timestamp", freq='360Min')]).size()
df_grouped_360_hex_8 = df_grouped_360_hex_8.reset_index().rename(columns={0: "trip_count"})

In [6]:
# 24 Hour = 1440 Mins
df_grouped_1440_hex_8 = df.groupby([pd.Grouper(key="hex_ID_pickup"),
                        pd.Grouper(key="trip_start_timestamp", freq='1440Min')]).size()
df_grouped_1440_hex_8 = df_grouped_1440_hex_8.reset_index().rename(columns={0: "trip_count"})

In [7]:
df_grouped_60_hex_8.to_parquet(Path("../../data/output/grouped/df_grouped_60_hex_8.parquet"))
df_grouped_120_hex_8.to_parquet(Path("../../data/output/grouped/df_grouped_120_hex_8.parquet"))
df_grouped_360_hex_8.to_parquet(Path("../../data/output/grouped/df_grouped_360_hex_8.parquet"))
df_grouped_1440_hex_8.to_parquet(Path("../../data/output/grouped/df_grouped_1440_hex_8.parquet"))

In [8]:
df_grouped_60_hex_8

,hex_ID_pickup,trip_start_timestamp,trip_count
0,882664520bfffff,2013-08-12 13:00:00,2
1,8826645219fffff,2013-01-01 09:00:00,1
2,8826645219fffff,2013-01-01 10:00:00,1
3,8826645219fffff,2013-01-01 11:00:00,4
4,8826645219fffff,2013-01-01 12:00:00,6
...,...,...,...
377168,88275936bbfffff,2013-06-13 10:00:00,2
377169,88275936bbfffff,2013-06-27 18:00:00,1
377170,88275936bbfffff,2013-12-08 18:00:00,1
377171,88275936d3fffff,2013-08-08 14:00:00,1
